## LSTM Part-of-Speech Tagger
### Importing the <code>torch</code> libraries

In [61]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import glob, os

torch.manual_seed(1)

### Helper functions...

In [224]:
# Read the data file
def read_data(filepath):
    data = []
    with open(filepath, 'r', encoding="utf8") as f:
        for line in f:
            data.append(tuple(zip(*[wt.split('/') for wt in line.strip().split()])))
    return data

# Get vocabs and tagset
def get_vocab_tagset(data):
    token_size, vocabs, tagset = 0, set(), set()
    for words, tags in data:
        token_size += len(words)
        vocabs.update(words)
        tagset.update(tags)
    return vocabs, tagset, token_size

# convert the sequencies to indexes and tensors
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

def to_index(aList):
    elem_to_idx = {e:aLest.index(i) for e in aLest}
    idx_to_elem = {i:e for e,i in elem_to_idx.items()}
    return elem_to_idx, idx_to_elem

### Initialisations

In [254]:
tagged_files = [
#         "igbo/ITC5.coarse*", "igbo/fiction.coarse*", #coarse
#         "igbo/ITC5.fine*", "igbo/fiction.fine*", #fine
        "welsh/*pos_coarse*",
#         "welsh/*sem_coarse*", 
#         "welsh/*both_fine*", "welsh/*both_coarse*",
#         "english/corpus.small"
#         "swedish/*coarse*"
       ]
for file in tagged_files: 
    training_data = []
    for file in glob.glob(os.path.join('all_corpora', file)):
        print(f"\nReading {file} ...", end='')
        training_data.extend(read_data(file))
        if len(training_data) >= 1000:
            break
    vocabs, tagset, token_size = get_vocab_tagset(training_data)
    print(f"\nSentences:\t{len(training_data)}\nToken size:\t{token_size}\nVocab size:\t{len(vocabs)}\nTagset size:\t{len(tagset)}")

word_to_idx, idx_to_word = to_index(list(vocabs))
tag_to_idx, idx_to_tag = to_index(list(tagset))


Reading all_corpora/welsh/cy_pos_coarse_tagged ...
Sentences:	611
Token size:	14875
Vocab size:	3897
Tagset size:	15


In [252]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'Adf',
 'Ans',
 'Ar',
 'Atd',
 'B',
 'Cys',
 'E',
 'Ebych',
 'Gw',
 'Rha',
 'Rhi',
 'U',
 'YFB',
 'pron',
 'unk'}

In [ ]:

# training_data = [
#     ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
#     ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
# ]

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.


In [11]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [15]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()

print("Scores before training:")
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
print("\nScores after training:")
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

Scores before training:
tensor([[-1.1153, -0.8478, -1.4113],
        [-1.1129, -0.8852, -1.3518],
        [-1.0766, -0.9060, -1.3660],
        [-1.1870, -0.8987, -1.2456],
        [-1.1577, -0.8997, -1.2762]])

Scores before training:
tensor([[-0.0307, -3.5960, -5.8840],
        [-4.1046, -0.0832, -2.7595],
        [-4.8994, -2.2309, -0.1220],
        [-0.0534, -3.0170, -5.8000],
        [-3.8753, -0.0462, -3.7117]])


In [78]:
# import numpy as np
for w, preds in list(zip(training_data[0][0], tag_scores)):
    preds = list(np.array(preds))
    idx = preds.index(max(preds))
    print(f"{w}/{ix_2_tag[idx]}", end=" ")

The/DET dog/NN ate/V the/DET apple/NN 

In [198]:
with open('all_corpora/welsh/cy_both_coarse_tagged', 'w', encoding='utf8') as f1:
    with open('all_corpora/welsh/cy_both_fine_tagged', 'r', encoding='utf8') as f2:
        i=0
        for line in f2:
            words, tags = tuple(zip(*[wt.split('/') for wt in line.strip().split()]))
            pos_tags = [tag.split('|')[0] for tag in tags]
            sem_tags = [re.findall(r'[A-Za-z]+\d*',tag.split('|')[1])[0] for tag in tags]
            coarse_pos_sem = ["|".join(ps) for ps in zip(pos_tags,sem_tags)]
            word_coarse_pos_sem = [f"{w}/{t}" for w,t in zip(words, coarse_pos_sem)]
#             print(f"{words}\n\n{tags}\n\n{both}\n\n{' '.join(word_both_tags)}")
            i+=1
#             print(f"{i}\t{' '.join(word_coarse_pos_sem)}")
            f1.write(f"{' '.join(word_coarse_pos_sem)}\n")
#             break